In [1]:
import torch
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from rdkit import Chem
#from dgllife.utils import smiles_to_bigraph
#from dgllife.utils import mol_to_bigraph
from rdkit.Chem import AllChem
from ase import Atoms
from ase.io import read
from ase.calculators.mopac import MOPAC

In [3]:
datasets = ['H']
for dataset in datasets:
    smiles = pd.read_csv('../../../Data/Solubility/dataset-' + dataset + '.csv')['SMILES']
    pot_energy = []
    for smile in tqdm(smiles):
        
        mol = Chem.MolFromSmiles(smile)
        mol = Chem.AddHs(mol)
        AllChem.EmbedMolecule(mol) 
        Chem.rdmolfiles.MolToXYZFile(mol,'tmp.xyz')
        try:
            mol = read('tmp.xyz')
            mol.calc = MOPAC(label='TMP', task='UHF BONDS GRADS')
            pot_energy.append(mol.get_potential_energy())
        except:
            pot_energy.append(None)
        
    out = pd.concat([pd.Series(smiles), pd.Series(pot_energy)], axis = 1)
    out.columns = ['SMILES', 'E_0']
    out.to_csv('dataset-%s.csv' % dataset, index = False)

  0%|          | 0/578 [00:00<?, ?it/s]